# Validate your predictions

In [99]:
import numpy as np
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.collocations import *
from nltk import FreqDist, word_tokenize
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


**In the cell below, please set the `predictions_path` variable to the path for your predictions file.**

In [100]:
with open('X_test.pkl', 'rb') as file:
    X_test = pickle.load(file)
    
with open('X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)
    
with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)

# Preprocessing

In [101]:
#helper function to tokenize, convert to lower case and remove stopwords
def process_article(speech, stopwords_list):
    tokens = nltk.word_tokenize(speech)
    stopwords_removed = [ token.lower() for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed

In [102]:
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']
speech_list_clean = [process_article(speech, stopwords_list) for speech in X_train]

In [105]:
# Vectorizing with TF-IDF
total_vocab = set()

for comment in speech_list_clean:
    total_vocab.update(comment)
len(total_vocab)

# Exploring Data With Frequency Distributions
articles_concat = []
for article in speech_list_clean:
    articles_concat += article
    
articles_freqdist = FreqDist(articles_concat)
articles_freqdist.most_common(200)   

vectorizer = TfidfVectorizer()
tf_idf_data_train = vectorizer.fit_transform(articles_freqdist)
#tf_idf_data_test = vectorizer.transform(newsgroups_test)
rf = RandomForestClassifier(n_estimators=100)
rf.fit(tf_idf_data_train, y_train)

# mow we have sparse matrix, find non-zero columns
non_zero_cols = tf_idf_data_train.nnz / float(tf_idf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tf_idf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

ValueError: Found input variables with inconsistent numbers of samples: [7953, 36]

Run this cell below to validate your predictions.

In [6]:
with open(predictions_path, 'rb') as file:
    predictions = pickle.load(file)
    
assert len(predictions) == 22, 'Predictions do not have the correct shape.'

assert np.all([type(x) == str for x in predictions]), 'Each prediction should be a string datatype.'

print('Your predictions passed the tests!')

IsADirectoryError: [Errno 21] Is a directory: '.'